In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
DATA='../Quotes/'

In [19]:
attributes = pd.read_parquet(DATA+"speaker_attributes.parquet.zip")

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [21]:
with pd.read_json(DATA+"quotes-2020.json.bz2",compression='bz2',lines=True,chunksize=1_000_000) as df_reader:
    for chunk in df_reader:
        processed_chunk = clean_data(chunk)
        processed_chunk.to_csv(DATA+"quotes-2020-cleaned.bz2",compression='bz2',mode='a',index=False,header=False)


KeyboardInterrupt: 

In [ ]:
def clean (chunk_list):
    chunk_list = chunk_list.drop_duplicates(subset="id", keep="first")
    chunk_list=chunk_list(chunk_list.genre!=NaN)
    chunk_list=chunk_list(chunk_list.age<100)
    #si le métier de la personne est compris dans sonn nom (du style de président), le retirer serait pas mal pour ensuite normer les noms

    